# Prediction Insights

## Overview

Howso Engine enables powerful predictions with complete attribution and detailed explanations to make learning from
and debugging your data and predictions as easy as possible. For more information on predictions with Howso Engine,
check out the [predictions user guide](https://docs.howso.com/user_guide/basic_capabilities/predictions.html).

In [1]:
from pprint import pprint

import pandas as pd
import plotly.graph_objects as go
import plotly.express as px

from howso.engine import Trainee
from howso.utilities import infer_feature_attributes

## Setup

This recipe will focus on the insights gained from predictions. Specifically, insights from:

- Influential Cases,
- Feature Contributions,
- Residual and Similarity Conviction,
- and Categorical Action Probabilities.

The [basic workflow guide](https://docs.howso.com/user_guide/basic_capabilities/basic_workflow.html) covers creating a Trainee and performing a basic react in detail.

### Load Data and Create Trainee

In [2]:
df = pd.read_csv("../../data/iris/iris.tsv.gz", sep="\t", compression="gzip")
train_data = df.iloc[:-30]
new_data = df[~df.index.isin(train_data.index)]
features = infer_feature_attributes(train_data)

t = Trainee(features=features)

df

,Unnamed: 0,sepal length,sepal width,petal length,petal width,class
0,27,5.2,3.5,1.5,0.2,Iris-setosa
1,14,5.8,4.0,1.2,0.2,Iris-setosa
2,25,5.0,3.0,1.6,0.2,Iris-setosa
3,82,5.8,2.7,3.9,1.2,Iris-versicolor
4,97,6.2,2.9,4.3,1.3,Iris-versicolor
...,...,...,...,...,...,...
145,126,6.2,2.8,4.8,1.8,Iris-virginica
146,124,6.7,3.3,5.7,2.1,Iris-virginica
147,78,6.0,2.9,4.5,1.5,Iris-versicolor
148,125,7.2,3.2,6.0,1.8,Iris-virginica


### Train, Analyze, and React

In [3]:
t.train(train_data)
t.analyze()
t.react_into_features(similarity_conviction=True)

reaction = t.react(
    contexts=new_data,
    context_features=["sepal length", "sepal width", "petal length", "petal width"],
    action_features=["class"],
    details={
        "influential_cases": True,
        "similarity_conviction": True,
        "feature_contributions": True,
        "feature_residuals": True,
        "robust_influences": True,
        "robust_residuals": True,
        "local_case_feature_residual_convictions": True,
        "categorical_action_probabilities": True,
    }
)

Note that, unlike in the basic workflow guide, we include several `details` in the react call and predict multiple cases as opposed to one. These are what will enable the
insights that we're going to get after the predictions are made.

For more information, see [the API documentation for `Trainee.react()`](https://docs.howso.com/api_reference/_autosummary/howso.engine.html#howso.engine.Trainee.react).

### Inspect the Predictions

Howso Engine has high accuracy even on small datasets.

Here we see a comparison of the trained data (dots) and the predicted data (stars). This plot shows that the predictions align well with the trained data.

In [4]:
train_data = train_data.astype({"class": str})
predicted_data = pd.concat([new_data.reset_index(drop=True).drop(columns="class"), reaction["action"]], axis=1)
predicted_data = predicted_data.astype({"class": str})

cmap = px.colors.qualitative.D3
fig = go.Figure()
for i, (label, group) in enumerate(train_data.groupby("class")):
    fig.add_trace(go.Scatter(
        x=group["petal length"],
        y=group["petal width"],
        mode="markers",
        name=label,
        marker=dict(color=cmap[i], opacity=0.75),
        legendgroup="trained",
        legendgrouptitle_text="Trained class",
    ))

for i, (label, group) in enumerate(predicted_data.groupby("class")):
    fig.add_trace(go.Scatter(
        x=group["petal length"],
        y=group["petal width"],
        mode="markers",
        marker=dict(size=12, symbol="star", color=cmap[i], opacity=0.75),
        name=label,
        legendgroup="predicted",
        legendgrouptitle_text="Predicted class",
        hovertext=group.index,
    ))

fig.update_layout(
    xaxis_title="Petal Length",
    yaxis_title="Petal Width",
    width=1250,
    title="Trained and Predicted Values"
)
fig.show()

For categorical action features, the prediction can be further understood with the `categorical_action_probabilities` detail.  
This shows the probability of each possible value of the target feature. This information can highlight cases that are on the
border of two classes, like some of the above points are.  The closer a case gets to a class border, the more mixed the
categorical action probabilities may get.

In [5]:
pprint(reaction["details"]["categorical_action_probabilities"], compact=True)

[{'class': {'Iris-versicolor': 1}},
 {'class': {'Iris-versicolor': 0.18905255715302757,
            'Iris-virginica': 0.8109474428469724}},
 {'class': {'Iris-virginica': 1}}, {'class': {'Iris-virginica': 1}},
 {'class': {'Iris-versicolor': 0.39467527137753533,
            'Iris-virginica': 0.6053247286224647}},
 {'class': {'Iris-versicolor': 1}}, {'class': {'Iris-setosa': 1}},
 {'class': {'Iris-virginica': 1}},
 {'class': {'Iris-versicolor': 0.8056923228738796,
            'Iris-virginica': 0.1943076771261205}},
 {'class': {'Iris-versicolor': 1}}, {'class': {'Iris-virginica': 1}},
 {'class': {'Iris-versicolor': 1}}, {'class': {'Iris-setosa': 1}},
 {'class': {'Iris-versicolor': 0.20118174108616763,
            'Iris-virginica': 0.7988182589138324}},
 {'class': {'Iris-versicolor': 1}}, {'class': {'Iris-setosa': 1}},
 {'class': {'Iris-setosa': 1}},
 {'class': {'Iris-versicolor': 0.20467404978617543,
            'Iris-virginica': 0.7953259502138246}},
 {'class': {'Iris-virginica': 1}},
 {'

For example, if we look at case `4`, we see a mix of probabilities because this case is on the border of two classes.

In [6]:
reaction["details"]["categorical_action_probabilities"][4]

{'class': {'Iris-versicolor': 0.39467527137753533,
  'Iris-virginica': 0.6053247286224647}}

### Insight 1: Which Cases Contributed?

Howso provides complete attribution for any and all predictions, showing exactly which cases influenced each prediction.  This can be used to understand or debug predictions.  For instance,
we can inspect the influential cases for one of the cases (case `4`) that was on the decision boundary in the plot above.  We can see that there is a mix of class values and no cases stand
out in terms of influence weight:

In [7]:
inf_cases_4 = pd.DataFrame(reaction["details"]["influential_cases"][4])
inf_cases_4

,sepal width,.session_training_index,petal length,petal width,.influence_weight,sepal length,.session,class
0,2.8,99,5.1,1.5,0.213655,6.3,ff387109-df22-4c88-ac41-7daf26108348,Iris-virginica
1,2.5,107,4.9,1.5,0.205513,6.3,ff387109-df22-4c88-ac41-7daf26108348,Iris-versicolor
2,2.2,34,5.0,1.5,0.197338,6.0,ff387109-df22-4c88-ac41-7daf26108348,Iris-virginica
3,2.9,61,5.6,1.8,0.194332,6.3,ff387109-df22-4c88-ac41-7daf26108348,Iris-virginica
4,2.9,89,4.7,1.4,0.189162,6.1,ff387109-df22-4c88-ac41-7daf26108348,Iris-versicolor


Compare that to the influential cases for case `20`, which is firmly in the center of the solo cluster.  Here we see a single class value.

In [8]:
inf_cases_20 = pd.DataFrame(reaction["details"]["influential_cases"][20])
inf_cases_20

,sepal width,.session_training_index,petal length,petal width,.influence_weight,sepal length,.session,class
0,3.5,48,1.4,0.3,0.201279,5.1,ff387109-df22-4c88-ac41-7daf26108348,Iris-setosa
1,3.4,63,1.6,0.4,0.200774,5.0,ff387109-df22-4c88-ac41-7daf26108348,Iris-setosa
2,3.4,9,1.7,0.2,0.199690,5.4,ff387109-df22-4c88-ac41-7daf26108348,Iris-setosa
3,3.3,116,1.7,0.5,0.199320,5.1,ff387109-df22-4c88-ac41-7daf26108348,Iris-setosa
4,3.4,43,1.4,0.2,0.198937,5.2,ff387109-df22-4c88-ac41-7daf26108348,Iris-setosa


With the influential cases we can derive additional insights, such as identifying anomalous cases from within the influential cases using `similarity_conviction`.  
This can help to identify data that are making predictions more noisy or identify what type of data should be collected to improve predictive power in the future.

In [9]:
inf_case_indices = inf_cases_4[[".session", ".session_training_index"]].values.tolist()

anom_df = t.get_cases(
    case_indices=inf_case_indices,
    features=["sepal length", "sepal width", "petal length", "petal width", "class", "similarity_conviction"]
)

anom_df.sort_values(by="similarity_conviction")

,sepal length,sepal width,petal length,petal width,class,similarity_conviction
2,6.0,2.2,5.0,1.5,Iris-virginica,0.879342
1,6.3,2.5,4.9,1.5,Iris-versicolor,0.973026
0,6.3,2.8,5.1,1.5,Iris-virginica,0.975006
3,6.3,2.9,5.6,1.8,Iris-virginica,1.007449
4,6.1,2.9,4.7,1.4,Iris-versicolor,1.017580


In [10]:
inf_case_indices = inf_cases_20[[".session", ".session_training_index"]].values.tolist()

anom_df = t.get_cases(
    case_indices=inf_case_indices,
    features=["sepal length", "sepal width", "petal length", "petal width", "class", "similarity_conviction"],
)

anom_df.sort_values(by="similarity_conviction")

,sepal length,sepal width,petal length,petal width,class,similarity_conviction
3,5.1,3.3,1.7,0.5,Iris-setosa,0.973207
2,5.4,3.4,1.7,0.2,Iris-setosa,0.994487
1,5.0,3.4,1.6,0.4,Iris-setosa,1.004521
0,5.1,3.5,1.4,0.3,Iris-setosa,1.016682
4,5.2,3.4,1.4,0.2,Iris-setosa,1.017910


### Insight 2: Which Features Contributed?

In addition to providing attribution to cases, Howso also provides robust feature contributions to explain which features contributed to each prediction.
When inspecting the feature importances for case `13`,  we see that `petal width` and `petal length` provided the majority of the contribution to the 
prediction, whereas for case `20` `petal width` and `petal length` only have slightly higher contributions than the other features.  This could indicate 
that different features are more important for predicting different classes within this dataset and that focusing on those features would be prudent.

Identifying that certain cases that influenced a prediction are anomalous can help to explain low predictive power in certain parts of the data or other
issues that may appear during deployment.

In [11]:
fcs_13 = pd.DataFrame(reaction["details"]["feature_contributions"][13:14], index=[13])
fcs_20 = pd.DataFrame(reaction["details"]["feature_contributions"][20:21], index=[20])

display(fcs_13)
display(fcs_20)

,sepal width,petal length,petal width,sepal length
13,0.079042,0.195514,0.419412,0.084437


,sepal width,petal length,petal width,sepal length
20,0.121058,0.183682,0.189012,0.143866


### Insight 3: How Certain is the Prediction?

[Residuals](https://docs.howso.com/user_guide/basic_capabilities/residuals.html) can characterize the uncertainty of the data around the prediction.  This will tell us which features are hard to predict in the region of the data around each case we're predicting.
If a prediction is less accurate than expected, this can explain which features were noisy and may have contributed to the problem.

In [12]:
pd.DataFrame(reaction["details"]["feature_residuals"])

,sepal width,petal length,petal width,sepal length,class
0,0.213341,0.454186,0.204130,0.475645,0.351789
1,0.234754,0.316981,0.167296,0.241495,0.290979
2,0.059831,0.564382,0.250722,0.370931,0.333809
3,0.269335,0.292286,0.167771,0.222342,0.267428
4,0.157917,0.343299,0.214646,0.280330,0.256317
5,0.113950,0.138793,0.125395,0.458459,0.159672
6,0.177571,1.478290,0.550890,0.717479,0.253472
7,0.172538,0.317616,0.198354,0.304765,0.177819
8,0.290945,0.475141,0.154640,0.345465,0.255485
9,0.247320,0.324430,0.175283,0.259364,0.100000


We can also use [`residual conviction`](https://docs.howso.com/user_guide/basic_capabilities/conviction.html#prediction-residual-conviction) to determine which features
are uncertain in a scale-invariant manner, which can be useful if you wish to compare different features of different scales against each other.  Higher residual conviction
indicates something that is less surprising than expected and lower redsidual conviction indicates the opposite.

In [13]:
pd.DataFrame(reaction["details"]["local_case_feature_residual_convictions"])

,sepal width,petal length,petal width,sepal length,class
0,1.757405,0.791718,0.740819,0.767820,1.676937
1,0.467282,1.016373,0.757844,1.052846,0.907631
2,5.001652,1.151222,0.466650,0.861155,0.351285
3,0.496611,0.840158,0.915799,0.880870,0.765000
4,0.565830,0.594974,0.440434,0.995502,0.298578
5,0.824828,2.179642,3.335412,1.191549,2.662438
6,0.686215,0.613692,1.737639,0.669593,1.309871
7,0.466555,1.434782,1.861849,0.979907,0.544921
8,0.476153,1.375950,1.065456,0.684694,0.950221
9,0.499042,2.277693,0.995399,1.581366,1.128279


### Insight 4: How Anomalous are the Predicted Cases?

By getting the [`similarity conviction`](https://docs.howso.com/user_guide/basic_capabilities/conviction.html#similarity-conviction) of the cases that we predict, 
we can determine which of them are anomalous relative to the trained data. This could help to highlight cases that are unusually difficult or easy to predict, such as can be the case
when model drift occurs as a form of model monitoring.

In [14]:
pprint(reaction["details"]["similarity_conviction"], compact=True)

[1.260888948379563, 1.2270378847785401, 1.2288560316321502, 1.2006045338440114,
 1.1159803673108029, 1.255407396300345, 1.2458837947677048, 1.2190831470412962,
 1.1400027738550151, 1.2225250365058915, 1.275033894962074, 1.2005403924411866,
 1.2492526384626443, 1.2488527031434467, 1.2201968569595487, 1.2320085465083763,
 1.216302532333635, 1.26740555797108, 1.1470413028232644, 1.0113368799719389,
 1.206482924667829, 1.2543533931428588, 1.149024596330311, 1.2350756498903146,
 1.240831907398804, 1.271870086241497, 1.2424703407841664, 1.2416573112135683,
 1.1934821669961284, 1.2214339785351966]


In this case, there are no cases with a low similarity_conviction which indicates that there are no cases that we reacted to which are particularly
anomalous.